In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import PIL
from google.colab import drive
import zipfile

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.listdir('gdrive/MyDrive/Dataset')

['dataset_HDD_train_kecil1.zip', 'dataset_HDD_val_kecil1.zip']

In [4]:
path_train = os.path.join('gdrive', 'MyDrive', 'Dataset', 'dataset_HDD_train_kecil1.zip')
path_val = os.path.join('gdrive', 'MyDrive', 'Dataset', 'dataset_HDD_val_kecil1.zip')

file = zipfile.ZipFile(path_train)
file.extractall()
file.close()

myfile = zipfile.ZipFile(path_val)
myfile.extractall()
myfile.close()

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1 / 255,
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    zoom_range=0.2, 
    horizontal_flip=True,  
    brightness_range=(0.8, 1.2),  
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        'dataset_HDD_train_kecil1',  # This is the source directory for training images
        target_size=(600, 600),  # All images will be resized to 150x150
        batch_size=64,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        'dataset_HDD_val_kecil1',  # This is the source directory for training images
        target_size=(600, 600),  # All images will be resized to 150x150
        batch_size=64,
        class_mode='categorical')

Found 10504 images belonging to 26 classes.
Found 1274 images belonging to 26 classes.


In [7]:
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow.keras.applications

#local_weight = os.path.join('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

pre_trained = tensorflow.keras.applications.InceptionResNetV2(input_shape=(600,600,3),
                         include_top=False,
                         weights= 'imagenet')


219055592/219055592 [==============================] - 1s 0us/step


In [8]:
for layer in pre_trained.layers:
    layer.trainable = False

In [9]:
model = tf.keras.models.Sequential()
model.add(pre_trained)
model.add(layers.BatchNormalization(renorm=True))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(26, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 17, 17, 1536)     54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 17, 17, 1536)     10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               393472    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                        

In [14]:
h = model.fit(train_generator,
              validation_data=val_generator,
              steps_per_epoch=50,
              epochs = 10,
              verbose = 1)

Epoch 1/10
50/50 [==============================] - 434s 9s/step - loss: 2.9370 - accuracy: 0.1184 - val_loss: 2.5396 - val_accuracy: 0.1538
Epoch 2/10
50/50 [==============================] - 406s 8s/step - loss: 2.2225 - accuracy: 0.2678 - val_loss: 1.7825 - val_accuracy: 0.6154
Epoch 3/10
50/50 [==============================] - 407s 8s/step - loss: 1.4620 - accuracy: 0.5084 - val_loss: 1.3771 - val_accuracy: 0.7308
Epoch 4/10
50/50 [==============================] - 402s 8s/step - loss: 0.8549 - accuracy: 0.7234 - val_loss: 1.2096 - val_accuracy: 0.7308
Epoch 5/10
50/50 [==============================] - 397s 8s/step - loss: 0.4468 - accuracy: 0.8585 - val_loss: 1.2072 - val_accuracy: 0.7308
Epoch 6/10
50/50 [==============================] - 399s 8s/step - loss: 0.2492 - accuracy: 0.9272 - val_loss: 1.3004 - val_accuracy: 0.7308
Epoch 7/10
50/50 [==============================] - 407s 8s/step - loss: 0.1583 - accuracy: 0.9584 - val_loss: 1.4119 - val_accuracy: 0.7308
Epoch 8/10
50

In [20]:
from tensorflow.keras.utils import load_img, img_to_array

tes_path = os.path.join('1copy1.jpg')
img = load_img(tes_path, target_size=(600, 600))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
yhat = model.predict(images)
yhat = np.argmax(yhat, axis=1)[0]
print(yhat)

1/1 [==============================] - 0s 44ms/step
12


In [21]:
loss_train = h.history(['train_loss'])
loss_val = h.history(['val_loss'])
train_acc = h.history(['acc'])
val_acc = h.history(['val_acc'])
epochs = range(1, 11)

TypeError: ignored

In [ ]:
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, train_acc, 'g', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [1]:
model.save('model_HDD.h5')

NameError: ignored